# Practice importing two data sets, joining with SQL, and analyzing data
Using 2 data sets from my research, I combined the data to run regression analyses. Skills practiced:

- Reading in dataset
- Dealing with missing data (continuous and categorical)
- Merging two datasets using SQL
- Collapsing the Gender variable to have 3 categories using SQL
- Recoding Gender 
- Running simple linear regression using statsmodel while controlling for age and gender
- Graphing linear regression

In [35]:
# Changing the Current Working Directory
import os
# Change the Current Working Directory
# Specify the new directory path
new_directory = '/Users/mickey.rice/Desktop'

# Change the current working directory
os.chdir(new_directory)

# Verify the change
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

Current Working Directory: /Users/mickey.rice/Desktop


## Load in the datasets 

In [51]:
# Load the PMD CSV File
import pandas as pd
csv_file_path = 'PMD.csv'
PMD_csv = pd.read_csv(csv_file_path)
print("PMD:")
display(PMD_csv.head())

PMD:


,ID,Accuracy,RT,PHQ9,CESD,DASS,PRMQ1,Confidence,JOL,N300,PP,Age,Sex,Gender,Race,Ethnicity,Handedness
0,PMD02,73.611111,1116.633001,16,33,7,27,11,60,-7.268333,2.764000,NaN,Female,Female,\te. White,\tb. Not Hispanic or Latino,Right handed
1,PMD03,76.388889,805.957503,13,26,0,28,8,83,-9.697000,0.866000,18.0,Female,Female,e. White,b. Not Hispanic or Latino,Right handed
2,PMD04,77.777778,713.443193,13,37,14,27,8,95,-1.530000,-4.111333,NaN,Male,Transgender Nonbinary,\te. White,\tb. Not Hispanic or Latino,Right handed
3,PMD05,68.055556,1137.573536,16,40,16,37,22,87,-2.507000,0.486333,18.0,Female,Female,\te. White,\tb. Not Hispanic or Latino,Right handed
4,PMD06,18.055556,796.787629,10,32,10,27,12,70,-2.468333,2.281333,18.0,Female,Female,e. White,b. Not Hispanic or Latino,Right handed


In [52]:
# Load the DEX CSV File
import pandas as pd
csv_file_path = 'DEX.csv'
DEX_csv = pd.read_csv(csv_file_path)
print("DEX:")
display(DEX_csv.head())

DEX:


,ID_Number,Q3.1_1,Q3.1_2,Q3.1_3,Q3.1_4,Q3.1_5,Q3.1_6,Q3.1_7,Q3.1_8,Q3.1_9,...,Q3.1_17,Q3.1_18,Q3.1_19,Q3.1_20,DEXTOTAL,F1,F2,F3,F4,F5
0,PMA01,5,2,1,2,5,4,2,1,4,...,1,5,4,1,56,5,14,11,8,6
1,PMA02,3,1,1,2,2,1,2,3,2,...,2,1,3,1,41,6,5,7,7,5
2,PMA03,1,4,3,4,5,3,4,2,4,...,3,4,4,2,66,9,8,12,12,10
3,PMA04,2,1,3,1,2,1,1,1,3,...,2,2,2,1,35,3,5,7,4,6
4,PMA06,4,2,5,3,5,5,4,3,3,...,3,5,4,1,69,9,14,15,11,6


## Dealing with missing data

In [53]:
import numpy as np
PMD_csv.isnull().sum()  # Returns the count of missing values in each column

ID            0
Accuracy      0
RT            0
PHQ9          0
CESD          0
DASS          0
PRMQ1         0
Confidence    0
JOL           0
N300          0
PP            0
Age           2
Sex           0
Gender        0
Race          1
Ethnicity     0
Handedness    0
dtype: int64

In [54]:
# Fill in the missing data with mean of the column
# I would prefer to use interpolation, but since the first participant is missing their age, I cannot use interpolation due to boundary issues
PMD_csv['Age'].fillna(PMD_csv['Age'].mean(), inplace=True)

# Fill missing values for Race with the Unknown
PMD_csv['Race'] = PMD_csv['Race'].fillna('Unknown')

# Check if there are any missing values left
PMD_csv.isnull().sum()

ID            0
Accuracy      0
RT            0
PHQ9          0
CESD          0
DASS          0
PRMQ1         0
Confidence    0
JOL           0
N300          0
PP            0
Age           0
Sex           0
Gender        0
Race          0
Ethnicity     0
Handedness    0
dtype: int64

In [55]:
import numpy as np
DEX_csv.isnull().sum()  # Returns the count of missing values in each column

ID_Number    0
Q3.1_1       0
Q3.1_2       0
Q3.1_3       0
Q3.1_4       0
Q3.1_5       0
Q3.1_6       0
Q3.1_7       0
Q3.1_8       0
Q3.1_9       0
Q3.1_10      0
Q3.1_11      0
Q3.1_12      0
Q3.1_13      0
Q3.1_14      0
Q3.1_15      1
Q3.1_16      0
Q3.1_17      0
Q3.1_18      0
Q3.1_19      0
Q3.1_20      0
DEXTOTAL     0
F1           0
F2           0
F3           0
F4           0
F5           0
dtype: int64

In [56]:
# Fill in the missing data with interpolation for numerical columns
DEX_csv['Q3.1_15'].interpolate(method='linear', inplace=True)

# Check if there are any missing values left
DEX_csv.isnull().sum()

ID_Number    0
Q3.1_1       0
Q3.1_2       0
Q3.1_3       0
Q3.1_4       0
Q3.1_5       0
Q3.1_6       0
Q3.1_7       0
Q3.1_8       0
Q3.1_9       0
Q3.1_10      0
Q3.1_11      0
Q3.1_12      0
Q3.1_13      0
Q3.1_14      0
Q3.1_15      0
Q3.1_16      0
Q3.1_17      0
Q3.1_18      0
Q3.1_19      0
Q3.1_20      0
DEXTOTAL     0
F1           0
F2           0
F3           0
F4           0
F5           0
dtype: int64

## Create SQLite database

In [57]:
import sqlite3

# Create a connection to an in-memory SQLite database
conn = sqlite3.connect(':memory:')

## Load DataFrames into the SQLite Database

In [58]:
# Write the dataframes to the SQLite database
PMD_csv.to_sql('table1', conn, index=False, if_exists='replace')
DEX_csv.to_sql('table2', conn, index=False, if_exists='replace')

136

## Perform the SQL Merge Query

In [59]:
# Perform the SQL merge query
query = """
SELECT ID, Accuracy, PHQ9, DEXTOTAL, F1, F2, F3, F4, F5, Age, Gender
FROM table1
JOIN table2
ON table1.ID = table2.ID_Number
"""

# Execute the query and fetch the result into a pandas dataframe
merged_df = pd.read_sql_query(query, conn)

# Display the merged dataframe
print(merged_df)

       ID   Accuracy  PHQ9  DEXTOTAL  F1  F2  F3  F4  F5        Age  \
0   PMD02  73.611111    16        53   9   8   6   8   5  19.692308   
1   PMD03  76.388889    13        70  13  14  12  10   4  18.000000   
2   PMD04  77.777778    13        59   8  10  11  11   7  19.692308   
3   PMD05  68.055556    16        85  15  15  10  13  13  18.000000   
4   PMD06  18.055556    10        53   5  10   9   9   7  18.000000   
5   PMD08  94.444444    10        36   5   6   6   4   8  18.000000   
6   PMD09  90.277778    16        38  11   5   5   7   4  18.000000   
7   PMD10  27.777778    23        71  10  12   9  13  10  19.000000   
8   PMD11  87.500000     3        39   5   8   5   4   7  22.000000   
9   PMD12  93.055556    11        49   6  12   7   7   6  18.000000   
10  PMD16  22.222222    14        52   6  13   7   9   5  18.000000   
11  PMD17  62.500000     3        41   5   8   4   6   5  18.000000   
12  PMD18  90.277778     2        38   4   8   6   7   5  18.000000   
13  PM

## Categorize gender into three groups: male, female, transgender

In [60]:
query = """
SELECT *, 
   CASE 
        WHEN table1.Gender IN ('Transgender Female/Trans Woman/MTF', 'Gender Queer', 'Nonbinary', 'Transgender Male/Trans Man/FTM', 'Transgender Nonbinary') THEN 'Transgender'
        ELSE table1.Gender
    END AS gender_collapsed
FROM table1
JOIN table2
ON table1.ID = table2.ID_Number
"""

# Execute the query and fetch the result into a pandas dataframe
df = pd.read_sql_query(query, conn)

# Display the  dataframe
print(df)

       ID   Accuracy           RT  PHQ9  CESD  DASS  PRMQ1  Confidence  JOL  \
0   PMD02  73.611111  1116.633001    16    33     7     27          11   60   
1   PMD03  76.388889   805.957503    13    26     0     28           8   83   
2   PMD04  77.777778   713.443193    13    37    14     27           8   95   
3   PMD05  68.055556  1137.573536    16    40    16     37          22   87   
4   PMD06  18.055556   796.787629    10    32    10     27          12   70   
5   PMD08  94.444444   836.034642    10    38     6     23          13   65   
6   PMD09  90.277778  1002.515621    16    36     5     22          18   75   
7   PMD10  27.777778  1219.304303    23    48    18     38          20   80   
8   PMD11  87.500000  1139.404717     3    28     3     25          15   60   
9   PMD12  93.055556  1030.054932    11    29    11     27           8   78   
10  PMD16  22.222222   847.326715    14    43    12     27          10   90   
11  PMD17  62.500000  1400.999523     3    29     3 

## Simple linear regression - controlling for age and gender
Using statsmodel

In [61]:
# load the necessary packages
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler

In [62]:
# Code categorical variable (Gender) using one-hot encoding
df = pd.get_dummies(df, columns=['gender_collapsed'], drop_first=True)

# Display the first few rows to see the changes
df.head()

,ID,Accuracy,RT,PHQ9,CESD,DASS,PRMQ1,Confidence,JOL,N300,...,Q3.1_19,Q3.1_20,DEXTOTAL,F1,F2,F3,F4,F5,gender_collapsed_Male,gender_collapsed_Transgender
0,PMD02,73.611111,1116.633001,16,33,7,27,11,60,-7.268333,...,4,1,53,9,8,6,8,5,0,0
1,PMD03,76.388889,805.957503,13,26,0,28,8,83,-9.697000,...,5,1,70,13,14,12,10,4,0,0
2,PMD04,77.777778,713.443193,13,37,14,27,8,95,-1.530000,...,4,1,59,8,10,11,11,7,0,1
3,PMD05,68.055556,1137.573536,16,40,16,37,22,87,-2.507000,...,4,1,85,15,15,10,13,13,0,0
4,PMD06,18.055556,796.787629,10,32,10,27,12,70,-2.468333,...,4,2,53,5,10,9,9,7,0,0


### Regression using the sm.OLS approach

In [65]:
# Calculate the p-value
# Use statsmodel to calculate the p-value

# Separate continuous and categorical variables
continuous_vars = df[['DEXTOTAL', 'Age']]
categorical_vars = df[['gender_collapsed_Male', 'gender_collapsed_Transgender']]

# Standardize continuous variables
scaler = StandardScaler()
continuous_vars_scaled = scaler.fit_transform(continuous_vars)

# Combine scaled continuous variables and categorical variables
X_scaled = pd.DataFrame(continuous_vars_scaled, columns=['DEXTOTAL', 'Age'])
X_scaled = pd.concat([X_scaled, categorical_vars.reset_index(drop=True)], axis=1)

# Add constant to predictors
X_scaled = sm.add_constant(X_scaled)

# Target variable
y = df['Accuracy']

# Fit OLS model
model = sm.OLS(y, X_scaled)
results = model.fit()

# Print summary
print(results.summary())


                            OLS Regression Results                            
Dep. Variable:               Accuracy   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     1.854
Date:                Wed, 10 Jul 2024   Prob (F-statistic):              0.134
Time:                        12:09:18   Log-Likelihood:                -243.18
No. Observations:                  53   AIC:                             496.4
Df Residuals:                      48   BIC:                             506.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const           

### Regression using the smf.OLS approach

In [66]:
# Calculate the p-value
# Use statsmodel to calculate the p-value

# Standardize relevant predictors and add them back to the DataFrame
scaler = StandardScaler()
df[['DEXTOTAL', 'Age']] = scaler.fit_transform(df[['DEXTOTAL', 'Age']])

# Define the regression formula
formula = 'Accuracy ~ DEXTOTAL + Age + gender_collapsed_Male + gender_collapsed_Transgender'

# Fit OLS model
model = smf.ols(formula=formula, data=df)
results = model.fit()

# Print summary
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:               Accuracy   R-squared:                       0.134
Model:                            OLS   Adj. R-squared:                  0.062
Method:                 Least Squares   F-statistic:                     1.854
Date:                Wed, 10 Jul 2024   Prob (F-statistic):              0.134
Time:                        12:10:30   Log-Likelihood:                -243.18
No. Observations:                  53   AIC:                             496.4
Df Residuals:                      48   BIC:                             506.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept       

The smf.ols approach is simpler to use becuase it handles the inclusion of the intercept term automatically based on the formula.
For sm.OLS, you need to standardize the predictors separately, and then ensure that the constant term is appropriately handled.

## Plot the data